### Abrindo o vídeo e mostrando o vídeo

In [1]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from IPython import display
from time import sleep

import pims
from PIL import Image

v = pims.Video('../dataset/25_20160407_front/normal/crowd/2016_04_07_06_49_39FrontColor.avi')

In [2]:
def play(video):
    fig = plt.figure()
    ax = fig.gca()
    fig.show()

    for ii in range(len(video)):
        im = video[ii]
        plt.imshow(im, cmap='gray')
        ax.set_title(ii)

        fig.canvas.draw()
        sleep(0.1)

In [3]:
def play_cmp(video1, video2):
    fig = plt.figure(figsize=(18,9))
    fig1 = fig.add_subplot(121)
    fig2 = fig.add_subplot(122)
    ax = fig.gca()
    fig.show()

    for ii in range(len(video1)):
        im1 = video1[ii]
        im2 = video2[ii]
        fig1.imshow(im1, cmap='gray')
        fig2.imshow(im2, cmap='gray')
        ax.set_title(ii)

        fig.canvas.draw()
        sleep(0.1)

Algumas informações sobre o vídeo aberto:

In [4]:
print(v.get_metadata())

v_array = np.array(v)
print('\n', v_array.shape)

{'plugin': 'ffmpeg', 'nframes': 322, 'ffmpeg_version': 'N-86111-ga441aa90e8-static http://johnvansickle.com/ffmpeg/ built with gcc 5.4.1 (Debian 5.4.1-8) 20170304', 'fps': 25.0, 'source_size': (320, 240), 'size': (320, 240), 'duration': 12.88}

 (322, 240, 320, 3)


Os campos da *tupla* anterior significam o seguinte:
(Frames, Altura, Largura, Cores)

### Convertendo o vídeo para escala de cinza

In [5]:
def to_gray(video):
    frames, width, hight, color = video.shape
    video_gray = np.zeros((frames, width, hight), dtype='uint32')
    
    for ii in range(len(video)):
        r, g, b = Image.fromarray(video[ii]).split()
        
        r = np.array(r, dtype='uint32')
        g = np.array(g, dtype='uint32')
        b = np.array(b, dtype='uint32')
        
        video_gray[ii] = (r + g + b) / 3
        
    return video_gray

In [6]:
I = to_gray(v_array)
I.shape
# play(x[0:30])

(322, 240, 320)

#### Funções auxiliares

In [7]:
 def histogram(img, bins, range):
    h, be = np.histogram(img, bins=bins, range=range)
    w = range[-1]/bins
    bin_centers = be[1:]-(w/2)
    
    return bin_centers, h, w

In [8]:
def frame_analysis(frame_orig, frame_proc):
    tfig = plt.figure()

    tfig0 = plt.subplot2grid((4, 4), (0, 0), rowspan=2, colspan=2)
    tfig1 = plt.subplot2grid((4, 4), (0, 2), rowspan=2, colspan=2)
    tfig2 = plt.subplot2grid((4, 4), (2, 0), rowspan=2, colspan=4)
    
    tfig0.set_title('Frame original')
    tfig0.axis('off')
    tfig0.imshow(frame_orig, cmap='gray')

    tfig1.set_title('Frame subtraido')
    tfig1.axis('off')
    tfig1.imshow(frame_proc, cmap='gray')

    (bc, h , w) = histogram(frame_proc, 120, (0, 1.0))
    
    tfig2.bar(bc, h, width=w)
    
    tfig.show()

### Background Modeling Independent of Time

In [17]:
tao = 0.98  # Threshold

In [18]:
B_BMIT = I[0] # background model
D_BMIT = np.abs(I - B_BMIT) # background subtraction

# foreground mask
F_BMIT = D_BMIT.copy() 
F_BMIT = F_BMIT / F_BMIT.max()
F_BMIT[F_BMIT >= tao] = 1.0
F_BMIT[F_BMIT < tao] = 0

In [19]:
frame_analysis(I[47], D_BMIT[47])
frame_analysis(I[47], F_BMIT[47])

###  The Long Term Average Background Modeling

In [24]:
tao_LTAMBM = 0.2

# backgound model
B_LTAMBM = np.zeros(I.shape)
for k in range(1, len(I)):
    B_LTAMBM[k] = (1-1/k)*B_LTAMBM[k-1] + (1/k) *I[k]

D_LTAMBM = np.abs(I - B_LTAMBM) # background subtraction

# foreground mask
F_LTAMBM = D_LTAMBM.copy() 
F_LTAMBM = F_LTAMBM / F_LTAMBM.max()
F_LTAMBM[F_LTAMBM >= tao_LTAMBM] = 1.0
F_LTAMBM[F_LTAMBM < tao_LTAMBM] = 0

In [23]:
frame_analysis(I[47], D_LTAMBM[47])
frame_analysis(I[47], F_LTAMBM[47])

### Moving Average Backgound Modeling

In [27]:
tao_MABM = 0.2

# background modeling
W = 5
B_MABM = np.zeros(I.shape)
for k in range((W-1), len(I)-1):
    B_MABM[k] = (1/W)*np.sum(I[(k-(W-1)):(k+1),:,:], axis=0)

D_MABM = np.abs(I - B_MABM) # background subtraction

# foreground mask
F_MABM = D_MABM.copy() 
F_MABM = F_MABM / F_MABM.max()
F_MABM[F_MABM >= tao_MABM] = 1.0
F_MABM[F_MABM < tao_MABM] = 0

In [28]:
frame_analysis(I[47], D_MABM[47])
frame_analysis(I[47], F_MABM[47])